In [1]:
import os
import sys

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore

from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt
# %matplotlib qt
import seaborn as sns

import bk.load
import bk.compute
import bk.plot
import bk.signal


In [2]:
def filt_struct(Q,metadata,region):
    t = Q.index.values
    try:
        return nts.TsdFrame(t,Q.values[:,(metadata.Region == region)])
    except:
        print('could not compuet')
        return Q

def compute_state_vector(Q,states):
    state_vector = np.empty_like(Q.index)
    state_vector = np.empty_like(Q.index,dtype= 'object')
    for i,s in states.items():
        if i == 'sleep': continue
        s = s.drop_short_intervals(0.1)
        state_vector[s.in_interval(Q)>0] = i
    state_vector[np.where(state_vector == None)] = 'None'

    return state_vector


In [17]:
bk.load.current_session_linux(local_path='Rat08/Rat08-20130713')
neurons, metadata = bk.load.spikes()
states = bk.load.states()
states['sleep'] = states['sws'].union(states['Rem'])


t, Q = bk.compute.binSpikes(neurons, 1)
# Q = scipy.ndimage.gaussian_filter1d(Q, 1, axis=1)
Q_all = nts.TsdFrame(t, Q.T, time_units="s")
# Q_all = Q_all.restrict(states['Rem'])
# Q_all = Q_all.restrict(states['sleep'])



state_vector = compute_state_vector(Q_all,states)
# state_vector[[0,1,2,3,4]] = ['None','drowsy','sws','Rem','wake']

# lfp = bk.load.lfp_in_intervals(23,states['Rem'])
# lfp = bk.load.lfp_in_intervals(23,states['sleep'])

# lfp = bk.load.lfp(23)
# filt_lfp = bk.signal.passband(lfp,4,12)
# power, phase = bk.signal.hilbert(lfp)
# Q_phase = phase.realign(Q_all)

# ripples = bk.load.ripples()
# rip_int = nts.IntervalSet(ripples['start'].index.values,ripples['stop'].index.values)
# is_rip = rip_int.in_interval(Q_all)>0


Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [4]:
Q_power = power.realign(Q_all)

NameError: name 'power' is not defined

In [5]:
stru = ['Hpc', 'BLA', 'Pir']
for s in stru:
    Q = filt_struct(Q_all, metadata, s)
    C = np.corrcoef(Q.values.T)
#     plt.imshow(C,vmin = 0,vmax = 0.01)
    print(C.shape)
    eigen_vals, eigen_vecs = np.linalg.eig(C)
    proj = Q.values.dot(eigen_vecs[:, :3])

    df = {'PC1': proj[:, 0],
          'PC2': proj[:, 1],
          'PC3': proj[:, 2],
          # 'power': Q_power,
        #   'phase': Q_phase,
        #   'rip': is_rip,
          'state': state_vector}
    df = pd.DataFrame(df)
    df['time'] = df.index

    fig = plt.figure(figsize=(12, 8), dpi=130)
    # ax = fig.add_subplot(111)
    sns.scatterplot(x='PC1', y='PC2', data=df, hue='power',
                    edgecolor='white', s=10,alpha= 0.9,hue_norm=(0,100000))
    # ax.scatter(df['PC1'],df['PC2'],c = df['phase'])
    plt.title(s)
    plt.show()


(60, 60)


NameError: name 'Q_power' is not defined

In [16]:
stru = ['Hpc', 'BLA', 'Pir']
stru = ['BLA']
%matplotlib qt
for s in stru:
    fig, ax = plt.subplot_mosaic('''
    A
    A
    A
    A
    B
    C''', figsize=(12, 12), constrained_layout=True)
    im = ax['A'].imshow(np.corrcoef(Q_all.values[:, metadata.Region == s]),
                        cmap='jet', vmin=0.2, vmax=1,aspect = 'auto')

    fig.colorbar(im, ax=ax['A'])
    # ax['B'].plot(state_vector)
    plt.sca(ax['B'])
    bk.plot.intervals(states['Rem'])
    bk.plot.intervals(states['sws'],'grey')

    ax['B'].sharex(ax['A'])

    # ax['C'].plot(k.labels_)
    # ax['C'].sharex(ax['A'])
    # ax['A'].set_aspect('auto')

    plt.title(s)


In [43]:
from tqdm import tqdm

In [ ]:
def comodulogram(lfp,amp_freq_min,amp_freq_max,phase_freq_min,freq_band,phase_nbins):

    amp_freq_vector = np.arange(amp_freq_min,amp_freq_max,freq_band)
    amp_env = []

    for i in amp_freq_vector:
        lfp_filt = bk.signal.passband(lfp,i,i+freq_band)
        amp_env.append(bk.signal.enveloppe(lfp_filt))

In [ ]:
comodulgram(lfp,0,100,0,100,2,18)

In [55]:
# from functools import lru_cache
# @lru_cache
def comodulgram(lfp,fmin,fmax,f_bins_size,phase_nbins):
    bins = np.linspace(-np.pi,np.pi,phase_nbins)
    bins = list(zip(bins,np.roll(bins,-1)))[:-1]
    nbins = len(bins)

    freq_bins = np.arange(fmin,fmax,f_bins_size)

    comod = np.empty((len(freq_bins),len(freq_bins)))
    for i,x in tqdm(enumerate(zip(freq_bins[:-1],np.roll(freq_bins,-1)))):
        filt_x = bk.signal.passband(lfp,x[0],x[1],order= 1)
        power_x, phase_x = bk.signal.hilbert(filt_x)
        for j,y in enumerate(zip(freq_bins[:-1],np.roll(freq_bins,-1))):  
            filt_y = bk.signal.passband(lfp,y[0],y[1],order= 1)
            env = bk.signal.enveloppe(filt_y)
            m = np.empty(len(bins))
            for k,(l,h) in enumerate(bins):
                m[k] = np.nanmean(env[(l<phase_x.values) & (phase_x.values<h)])
            comod[i,j] = c_mi(m,nbins)
    return comod

In [56]:
def c_mi(m,n):
    m = m/np.sum(m)

    H = -np.sum(m * np.log(m))
    Dkl = np.log(n) - H

    return Dkl / np.log(n)

In [18]:
import bk.load
import bk.signal
import numpy as np

bk.load.current_session_linux()
states = bk.load.states()
lfp = bk.load.lfp_in_intervals(23,states['Rem'])

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713


In [59]:
c = comodulgram(lfp,0.1,100,2,18)

22it [16:22, 44.67s/it]


KeyboardInterrupt: 

In [58]:
%matplotlib qt
import matplotlib.pyplot as plt
plt.imshow(c)

In [3]:
theta = bk.signal.passband(lfp,4,10)
gamma = bk.signal.passband(lfp,60,120)
power,phase = bk.signal.hilbert(theta)
env = bk.signal.enveloppe(gamma)

In [40]:
b = np.array(bins)[:,0]
# b = np.hstack((b,b+2*np.pi))
# m = np.hstack((m,m))
# plt.bar(b,m)